## Obtendo dados via crawler do portal OS Info

#### Carregando bibliotecas

In [ ]:
import os
import glob
import openpyxl
import requests
import pandas as pd
import json
from datetime import date

#### Funções auxiliares

In [ ]:
def listar_arquivos(extensao='xlsx'):
    search = "input/*.{}".format(extensao)
    files = glob.glob(search)
    files.sort()
    return files

In [ ]:
# curl
# 'https://osinfo.prefeitura.rio/report/groupedExpenses/server/groupedExpensesService/generateGroupExpensesReportByExpenses'
# -H 'Accept: application/json, text/javascript, */*; q=0.01'
# -H 'Referer: https://osinfo.prefeitura.rio/pages/application-container.html'
# -H 'Origin: https://osinfo.prefeitura.rio'
# -H 'X-Requested-With: XMLHttpRequest'
# -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
# -H 'Content-Type: application/json'
# --data-binary '
# {"cod_os":263,
#  "id_contrato":11,
#  "contas":[],
#  "tipo_unidade":null,
#  "unidades":[],
#  "tipo_relatorio":"EXPENSES",
#  "ref_inicio_mes":"01",
#  "ref_inicio_ano":"2012",
#  "ref_fim_mes":"09",
#  "ref_fim_ano":"2015"}'
# --compressed

In [ ]:
url = 'https://osinfo.prefeitura.rio/report/groupedExpenses/server/groupedExpensesService/generateGroupExpensesReportByExpenses'

In [ ]:
data = '{"cod_os":%s,"id_contrato":%s,"contas":[],"tipo_unidade":null,"unidades":[],"tipo_relatorio":"EXPENSES","ref_inicio_mes":"%s","ref_inicio_ano":"%s","ref_fim_mes":"%s","ref_fim_ano":"%s"}'

In [ ]:
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://osinfo.prefeitura.rio/pages/application-container.html',
    'Origin': 'https://osinfo.prefeitura.rio',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
    'Content-Type': 'application/json',
}

In [ ]:
arquivos = listar_arquivos()

#### Iniciando o processo - salvando os JSONs de retorno para cada contrato das OSS com a Prefeitura do Rio

In [ ]:
hoje = date.today()
mes = hoje.strftime("%m")
ano = hoje.strftime("%Y")
for arquivo in arquivos:
    df = pd.read_excel(arquivo, sheet_name='contratos')
    for _index, row in df.iterrows():
        cod_os = row['ORGANIZACAO'].split(' - ')[0]
        cod_contrato = row['COD_CONTRATO']
        dt_inicio = row['DATA_INICIO']
        dt_fim = row['DATA_FIM']
        try:
            ref_inicio_mes = row['DATA_INICIO'].month
            ref_inicio_ano = row['DATA_INICIO'].year
            ref_fim_mes = row['DATA_FIM'].month
            ref_fim_ano = row['DATA_FIM'].year
            if dt_fim <= dt_inicio:
                ref_fim_mes = mes
                ref_fim_ano = ano
            print("valores: {} - {} - {} - {} - {} - {}".format(cod_os, cod_contrato, ref_inicio_mes, ref_inicio_ano, ref_fim_mes, ref_fim_ano))
            payload = data % (cod_os, cod_contrato, ref_inicio_mes, ref_inicio_ano, ref_fim_mes, ref_fim_ano)
            response = requests.post(url, headers=headers, data=payload)
            with open("output/org_{}_contrato_{}.json".format(cod_os, cod_contrato), 'w') as fp:
                json.dump(response.json(), fp)
        except:
            print("Erro no contrato: {}".format(cod_contrato))
            continue